### OPEN API 를 이용한 챗봇 시스템 

In [2]:
!pip show openapi
# !pip install openai

Name: openapi
Version: 2.0.0
Summary: Python OpenAPI 2.0 (Swagger) object model
Home-page: https://github.com/globality-corp/openapi
Author: Globality Engineering
Author-email: engineering@globality.com
License: 
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: inflection, jsonschema
Required-by: 


---
### OPENAI API 호출 예제 

In [3]:
from openai import OpenAI
from pprint import pprint
import os
api_key=os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

client = OpenAI(api_key=api_key)
model = 'gpt-3.5-turbo-1106'

messages = [
    {'role':'system','content': 'You are a helphful assistant.'},
    {'role':'user','content': '군자역 혼잡도 알려줘?'}
]
response = client.chat.completions.create(model=model, messages=messages).model_dump()
pprint(response)

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'content': '죄송합니다, 현재 군자역의 혼잡도 정보를 파악할 수 있는 방법은 '
                                     '없습니다. 군자역의 혼잡도 정보는 해당 역에서 제공하는 공공 시설의 안내 '
                                     '시스템을 통해 확인하시거나, 현지에서 직접 확인하시는 것이 좋을 것 '
                                     '같습니다.',
                          'function_call': None,
                          'role': 'assistant',
                          'tool_calls': None}}],
 'created': 1720079963,
 'id': 'chatcmpl-9hBf9yOjwL0BLc331DxO3P85cDimS',
 'model': 'gpt-3.5-turbo-1106',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_44132a4de3',
 'usage': {'completion_tokens': 102, 'prompt_tokens': 36, 'total_tokens': 138}}


> gpt 는 json type 으로 답을 넘겨준다. 

---
### 

 * [model dump ]
    : 메서드 체이닝 -> 메서드를 연쇄적으로 호출하는 방식 

---
### ai chatbot 을 만들기 위해서는 프롬프트 엔지니어링이 필요하다.

In [4]:
prompt = """
내일 다음 주제로 미래 교육 포럼에서 발표할 발제문을 작성해야합니다.
발제문의 주요 내용을 불릿 기호로 간략히 정리해 주세요.
주제:{agenda}


""".format(agenda="인공지능 시대의 교육의 역활과 의미")
model = 'gpt-3.5-turbo-1106'
response = client.chat.completions.create(
    model= model,
    messages=[{'role':'user','content':prompt}]
).to_dict
pprint(response)



<bound method BaseModel.to_dict of ChatCompletion(id='chatcmpl-9hBfBsl4xnnmLE2BNdCMu7MJ7dVbS', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='- 인공지능 기술의 발전으로 교육의 역할이 변화하고 있는 현상 파악\n- 인공지능이 교육에 미치는 영향과 가능성에 대한 이해와 고찰\n- 학습자 중심의 개별 맞춤형 교육 시스템 구축을 통한 인공지능 기술의 적용 가능성\n- 교사의 역할과 역량 강화를 통한 인공지능과 협업하는 교육 환경 조성\n- 디지털 리터러시와 스마트 러닝 활성화를 통한 인공지능 시대 교육의 의미와 가치 재정립', role='assistant', function_call=None, tool_calls=None))], created=1720079965, model='gpt-3.5-turbo-1106', object='chat.completion', service_tier=None, system_fingerprint='fp_44132a4de3', usage=CompletionUsage(completion_tokens=215, prompt_tokens=97, total_tokens=312))>


In [6]:
from pprint import pprint

prompt = """
내일 다음 주제로 미래 교육 포럼에서 발표할 발제문을 작성해야합니다.
발제문의 주요 내용을 불릿 기호로 간략히 정리해 주세요.
주제:{agenda}
""".format(agenda="인공지능 시대의 교육의 역할과 의미")

model = 'gpt-3.5-turbo-1106'
response = client.chat.completions.create(
    model=model,
    messages=[{'role': 'user', 'content': prompt}]
)

pprint(response.choices[0].message.content)


('- 인공지능 기술의 발전은 교육에 어떤 영향을 미치는가?\n'
 '- 교육이 인공지능 시대에 더 중요한 이유는?\n'
 '- 인공지능과 함께 하는 교육의 가장 큰 장점은 무엇인가?\n'
 '- 교육과 인공지능이 상호 보완적으로 발전할 수 있는 방안은?')


---
### Chater 5 프롬프트 엔지니어링의 핵심 기법 
- few shot
- Cot
-SC
- ToT
- ReAct
-RAG


In [12]:
from pprint import pprint
import json
template = """
당신은 번역함수이며, 반환값은 반드시 JSON 데이터 여야 합니다. 
STEP 별로 작업을 수행하면서 그 결과를 아래의 출력 결과 JSON 포맷에 작성하세요.
STEP-1. 아래 세 개의 백틱으로 구분된 텍스트를 원문 그대로 읽어 올것
STEP-2. 입력받은 텍스트가 긍정적이라면 True를 표기할것
STEP-3. 다음의 말투로 번역할 것: ["지구의 나이는 45억살이야.","세종 대왕은 조선의 위대한 국왕이야."]
```{text}```
---
출력결과 : {{"STEP-1":<입력텍스트>, "STEP-2": <true/false>, "STEP-3":<번역결과>}}
"""
text = "William Shakespeare was an English playwrit, poet and actor.  He is widely regarded as the greatest writer in the English language and the world's pre-eminent dramatist."
template = template.format(text=text)
context = [{"role": "user","content": template}]

model_1 = 'gpt-3.5-turbo-1106'
model_2 = 'gpt-4-0613'
response = client.chat.completions.create(
    model=model_2,
    messages=context,
    temperature=0,
    top_p=0,
    seed=1234
).model_dump()
pprint(json.loads(response['choices'][0]['message']['content']))
# pprint(response.choices[0].message.content)


{'STEP-1': 'William Shakespeare was an English playwrit, poet and actor.  He '
           'is widely regarded as the greatest writer in the English language '
           "and the world's pre-eminent dramatist.",
 'STEP-2': True,
 'STEP-3': ['윌리엄 셰익스피어는 영국의 희곡 작가, 시인, 배우였어.',
            '그는 영어로 가장 위대한 작가로 널리 인정받고 있고, 세계에서 가장 뛰어난 극작가야.']}
